In [ ]:
!pip install --user --upgrade pixiedust

In [1]:
from pixiedust.packageManager import PackageManager
pkg=PackageManager()
pkg.installPackage("graphframes:graphframes:0",base="http://dl.bintray.com/spark-packages/maven",sc=sc)
pkg.printAllPackages()

Opened database successfully
Package already installed: graphframes:graphframes:0
Added Python Files
org.apache.commons:commons-csv:1.4 => /Users/dtaieb/data/libs/commons-csv-1.4.jar
graphframes:graphframes:0.1.0-spark1.6 => /Users/dtaieb/data/libs/graphframes-0.1.0-spark1.6.jar
com.databricks:spark-csv_2.10:1.4.0 => /Users/dtaieb/data/libs/spark-csv_2.10-1.4.0.jar
com.databricks:spark-avro_2.10:2.0.1 => /Users/dtaieb/data/libs/spark-avro_2.10-2.0.1.jar
ai.h2o:sparkling-water-core_2.10:1.6.5 => /Users/dtaieb/data/libs/sparkling-water-core_2.10-1.6.5.jar


In [6]:
from pixiedust.display import *
sqlContext = SQLContext(sc)
dd = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Hector", 75),
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Hector", 75),
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Hector", 75),
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Hector", 75),
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Hector", 75)
], ["id", "name", "age"])

display(dd)

a,Alice,34
b,Bob,36
c,Charlie,30
d,Hector,75
a,Alice,34
b,Bob,36
c,Charlie,30
d,Hector,75
a,Alice,34
b,Bob,36
c,Charlie,30


In [7]:
#import the Graphs example
from graphframes.examples import Graphs
#create the friends example graph
g=Graphs(sqlContext).friends()
#use the pixiedust display
display(g)

a,Alice,34
b,Bob,36
c,Charlie,30
d,David,29
e,Esther,32
f,Fanny,36


In [8]:
sc.addPyFile("https://github.com/ibm-cds-labs/simple-data-pipe-connector-flightstats/raw/master/flightPredict/training.py")
sc.addPyFile("https://github.com/ibm-cds-labs/simple-data-pipe-connector-flightstats/raw/master/flightPredict/run.py")
import training
import run
sqlContext=SQLContext(sc)
training.sqlContext = sqlContext
training.cloudantHost='http://127.0.0.1:5984'
training.cloudantUserName='dtaieb'
training.cloudantPassword='password'

airports = training.loadDataSet("flight-metadata", "airports")
print("airports count: " + str(airports.count()))
flights = training.loadDataSet("pycon_flightpredict_training_set","training")
print("flights count: " + str(flights.count()))

Successfully cached dataframe
Successfully registered SQL table airports
airports count: 17535
Successfully cached dataframe
Successfully registered SQL table training
flights count: 33336


In [13]:
from pyspark.sql import functions as f
from pyspark.sql.types import *
rdd = flights.flatMap(lambda s: [s.arrivalAirportFsCode, s.departureAirportFsCode]).distinct()\
    .map(lambda row:[row])
vertices = airports.join(
      sqlContext.createDataFrame(rdd, StructType([StructField("fs",StringType())])), "fs"
    ).dropDuplicates(["fs"]).withColumnRenamed("fs","id")
vertices.cache()
vertices.registerTempTable("vertices")
print(vertices.count())

422


In [14]:
edges=flights.withColumnRenamed("arrivalAirportFsCode","dst")\
    .withColumnRenamed("departureAirportFsCode","src")\
    .drop("departureWeather").drop("arrivalWeather").drop("pt_type").drop("_id").drop("_rev")
edges.cache()
edges.registerTempTable("edges")

In [15]:
from graphframes import GraphFrame
edges2 = sqlContext.sql("select src, dst, count(1) as count from edges group by src, dst")
g = GraphFrame(vertices, edges2)
display(g)

IAH,f4f2c5ee32a9328500ffc78e5a66d31a,1-8b538bb6029537264ed8394e7ed4d38e,True,Houston,HOU,1,US,United States,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/IAH?codeType=fs,97,IAH,IAH,KIAH,29.983333,2015-12-26T22:18:25.090,-95.34,airports,George Bush Intercontinental Airport,None,77032,North America,TX,16930 JFK Boulevard,,America/Chicago,-6.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/IAH?codeType=fs,TXZ213
STI,f4f2c5ee32a9328500ffc78e5a9a3e35,1-18a4364097bb966988993e0135cc9875,True,Santiago,STI,4,DO,Dominican Republic,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/STI?codeType=fs,564,None,STI,MDST,19.402928,2015-12-27T00:18:25.126,-70.601868,airports,Cibao International Airport,None,None,Caribbean,None,None,None,America/Santo_Domingo,-4.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/STI?codeType=fs,None
YUL,0f63c432b6755d966d7df23ed631b51f,1-c9694e9728790506117e54e1dc672772,True,Montreal,YMQ,1,CA,Canada,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/YUL?codeType=fs,117,None,YUL,CYUL,45.457715,2015-12-26T23:18:25.368,-73.749906,airports,Montreal-Pierre Elliott Trudeau International Airport,None,None,North America,QC,975 Romeo Vachon Street North,None,America/Toronto,-5.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/YUL?codeType=fs,None
GEG,f4f2c5ee32a9328500ffc78e5a452638,1-495cb240085b44eed2047e4b4ce54613,True,Spokane,GEG,2,US,United States,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/GEG?codeType=fs,2372,GEG,GEG,KGEG,47.625147,2015-12-26T20:18:25.065,-117.537638,airports,Spokane International Airport,None,99224,North America,WA,9000 West Airport Drive,,America/Los_Angeles,-8.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/GEG?codeType=fs,WAZ036
LEB,0f63c432b6755d966d7df23ed61c7a93,1-cd9d7f6c7b15e73508626e9468b47b78,True,Lebanon,LEB,4,US,United States,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/LEB?codeType=fs,604,LEB,LEB,KLEB,43.629359,2015-12-26T23:18:25.363,-72.310007,airports,Lebanon Regional Airport,None,03784,North America,NH,Airport Drive,,America/New_York,-5.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/LEB?codeType=fs,NHZ005
YUM,f4f2c5ee32a9328500ffc78e5a5f6459,1-7a4229296d774179dbad4c115aa96f2d,True,Yuma,YUM,3,US,United States,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/YUM?codeType=fs,216,NYL,YUM,KNYL,32.668605,2015-12-26T21:18:25.119,-114.599263,airports,Yuma International Airport,None,None,North America,AZ,None,None,America/Phoenix,-7.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/YUM?codeType=fs,AZZ025
TUS,f4f2c5ee32a9328500ffc78e5a6d9a08,1-cbfa78ce35ebcb2a6c417889413ab15e,True,Tucson,TUS,3,US,United States,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/TUS?codeType=fs,2643,TUS,TUS,KTUS,32.120688,2015-12-26T21:18:25.086,-110.937369,airports,Tucson International Airport,None,85706,North America,AZ,7005 South Plumer Ave.,,America/Phoenix,-7.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/TUS?codeType=fs,AZZ033
STL,f4f2c5ee32a9328500ffc78e5a6ad77e,1-aadfb29135d141fd9dca8957b7fa52e5,True,Saint Louis,STL,1,US,United States,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/STL?codeType=fs,540,STL,STL,KSTL,38.742279,2015-12-26T22:18:25.121,-90.365869,airports,Lambert-St. Louis International Airport,None,63145,North America,MO,10701 Lambert International Blvd.,None,America/Chicago,-6.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/STL?codeType=fs,MOZ063
ZCL,0f63c432b6755d966d7df23ed6150c39,1-ed7107a4f6e7552bf79085a408aef299,True,Zacatecas,ZCL,4,MX,Mexico,https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/ZCL?codeType=fs,7021,None,ZCL,MMZC,22.8,2015-12-26T22:18:25.370,-102.55,airports,La Calera Airport,None,None,North America,None,None,None,America/Mexico_City,-6.0,https://api.flightstats.com/flex/weather/rest/v1/json/all/ZCL?codeType=fs,None
BRL,f4f2

In [ ]:
from pyspark.sql.functions import *
display(g.degrees.sort(desc("degree")))

ATL,218
ORD,197
DFW,197
IAH,154
DEN,147
CLT,146
JFK,121
PHL,120
MIA,109
MSP,108
LAX,108


In [ ]:
#airport ranking
ranks = g.pageRank(resetProbability=0.20, maxIter=5)
display(ranks)

In [ ]:
#Breath First Search
paths = g.bfs(fromExpr="id='BOS'",toExpr="id = 'SFO'",maxPathLength = 1)
display(paths)

In [ ]:
query1 = g.find("(a)-[e]->(b);(b)-[e2]->(c);(c)-[e3]->(d)")
display(query1)

In [ ]:
components = g.stronglyConnectedComponents(maxIter=10).select("id","component").orderBy("component")
display(components)

In [ ]:
communities = g.labelPropagation(maxIter=5).select("id", "label")
display(communities)